# Calculate Q-Rookley

- Input data "20172022_processed_1_3_4.csv", from 2017/7/1 to 2022/12/31, same as used in Figlewski

- Use 500 rolling window for P density (when calculating EPK)

- Use logreturn instead of log+1

- Use 
```
r.source('Q_Rookley_main_3.R')
```
and
```
'EPK_library_1.r'
```
to calculate Q_Rookley, along with 
```
get_btc_prices_2015_2022
```
for physical density.

- Interest rate: 3-month treasury bond

- Moneyness range is [min moneyness, max moneyness]

- Bandwidth 0.3

- tau=8

In [1]:
import platform
plat = platform.system()
import locale
if plat == 'Darwin':
    print('Using Macos Locale')
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
else:
    locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import datetime
import pandas as pd
import sys
import matplotlib.pyplot as plt
import csv
import gc
import math
import numpy as np
import os
import pdb
import pickle
import statsmodels.api as sm
import time


import rpy2.robjects.packages as rpackages
from rpy2 import robjects

from scipy import stats
from scipy.integrate import simps
from scipy import interpolate

from src.helpers import save_dict, load_dict
from src.brc import BRC
from src.strategies import IronCondor
from src.quandl_data import get_btc_prices_2015_2022

from numpy.lib.function_base import insert

from statsmodels.nonparametric.kernel_regression import KernelReg as nadarajawatson

from sklearn.neighbors import KernelDensity

from pathlib import Path

def filter_sub(_df,  mindate, maxdate, tau):
    # Subset

    # Only calls, tau in [0, 0.25] and fix one day (bc looking at intra day here)
    # (_df['is_call'] == 1) & --> Dont need to filter for calls because that happens in the R script
    # Also need to consider put-call parity there
    sub = _df[(_df['date'] >= mindate) & (_df['date'] <= maxdate) & (_df['moneyness'] >= 0.5) & (_df['moneyness'] < 2) &(_df['mark_iv'] > 0)]# &

    if tau > 0:
        sub = sub[sub['tau'] == tau]

    nrows = sub.shape[0]
    if nrows == 0:
        raise(ValueError('Sub is empty'))
   
    sub['moneyness'] = round(sub['moneyness'], 3)
    sub['underlying_price'] = round(sub['underlying_price'], 2)

    sub = sub.drop_duplicates()
    
    print(sub.describe())

    if nrows > 50000:
        print('large df, subsetting')
        sub = sub.sample(10000)
        print(sub.describe())

    return sub

def prepare_confidence_band_data(df):

            # Old
                # Save for Confidence Band 
                    # Column 1 - Day
                    # Column 2 - Month
                    # Column 3 - Year
                    # Column 4 - Call Indicator
                    # Column 5 - Time Maturity in Days
                    # Column 6 - Strike
                    # Column 7 - IV
                    # Column 8 - Spot
                    # Column 9 - Risk Free Rate
                    # Column 10 - 
                    # Column 11 - Forward
                # 3	1	1997	0	49	3025	165.8	2861.706	0.031465	0.150026951	2863.26
                # Day Month Year Put Days Strike IV     Spot         Risk Free    Tau in Days (255)       Forward

    """
    # Mapping Table for Input

    # Column ... X
    # 1 - date
    # 2 - IV
    # 3 - 1 for Call, 0 for Put
    # 4 - Tau 
    # 5 - Strike
    # 6 - assigned meanwhile as put2call[,6]+ put2call[,7] -  put2call[,5]*exp(- mean(put2call[,8])* tau);
    # 7 - Spot
    # 8 - Probably Interest Rate
    # 9 - assigned meanwhile Moneyness: Spot/Strike
    """
    b = df[['date', 'mark_iv', 'putcall', 'tau', 'K', 'underlying_price', 'interest_rate', 'moneyness', 'instrument_price']]
    b['placeholder'] = 0
    b = b.reset_index(drop=True)
    # b['interest_rate'] = 0.00
    b['forward'] = 1

    # Reorder
    df_c = b[['date', 'mark_iv', 'putcall', 'tau', 'K', 'instrument_price', 'underlying_price', 'interest_rate', 'moneyness']]

    unique_maturities = df.days_to_maturity.unique()[0]
    fname = str('confidence_band_input_') + str(unique_maturities) + str('.csv')
    df_c.to_csv(fname, index = False)
    return fname

def bootstrap(conf_fname, phys_fname, rdate, tau, simmethod, r, out_dir, bw, lower, upper):
                            
    try:
        # MAKE SURE TO USE DIFFERENT 2ND INPUT FOR SP500!!
        moneyness, spd, epk, epk_lo, epk_up, logret, spd_logret, epk_logret, epk_logret_lo, epk_logret_up, epk_BS, tau, volas, spd_lo, spd_up, cdf_m, cdf_ret= r.bootstrap_epk(robjects.StrVector([conf_fname]), 
                                      robjects.StrVector([phys_fname]),
                                      robjects.StrVector(rdate),
                                      robjects.StrVector([out_dir]),
                                      robjects.FloatVector([bw]),
                                      robjects.FloatVector([lower]),
                                      robjects.FloatVector([upper])
                                     )
        # Can recover PD here as SPD/EPK
        spd_df = pd.DataFrame({'m': moneyness,
                               'spdy': spd,
                               'spdy_up': spd_up,
                               'spdy_dn': spd_lo,
                               'epk': epk,
                               'ret': logret,
                               'spd_ret': spd_logret,
                               'cb_ret_up': epk_logret_up,
                               'cb_ret_dn': epk_logret_lo,
                               'epk_ret': epk_logret,
                               'volatility': volas,
                               'epk_up': epk_up,
                               'epk_dn': epk_lo,
                               'cdf_m': cdf_m,
                               'cdf_ret': cdf_ret})

        return spd_df
    
    except Exception as e:
        print('Sim failed, proceeding with the next one... - ', simmethod)
        print('exception: ', e)
        
# Initiate R Objects
base = rpackages.importr("base") 
r = robjects.r
r.source('src/confidence_bands.R')
r.source('src/Q_Rookley_main_3.R')

# BTC prices
tdat = get_btc_prices_2015_2022()
tdat[::-1].to_csv('Data/BTC_USD_Quandl_2015_2022.csv', index = False)

bw_range = np.array([0.3])
r_bandwidth = bw_range[0]

simmethod = ['oldschool']

df = pd.read_csv('Data/20172022_processed_1_3_4.csv')
interest_rate_data = pd.read_csv('Data/IR_daily.csv')
interest_rate_data = interest_rate_data.rename(columns={'index': 'date','DTB3': 'interest_rate'})

df1=df[df['tau']==8]
dates=df1['date'].unique()
#df1=df[df['date']>=dates[9]]
#dates = df1['date'].unique()

dates1 = dates

error_i = []
for i0 in range(dates1.size):
    df2 = df1[df1['date']==dates1[i0]]
    
    curr_day = datetime.datetime.strptime(dates1[i0],'%Y-%m-%d')
    curr_day_starttime = curr_day.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
    curr_day_endtime = curr_day.replace(hour = 23, minute = 59, second = 59, microsecond = 0)
    print(curr_day)

    df3 = pd.merge(df2, interest_rate_data, on='date', how='left')
    

    df3['interest_rate'] = df3['interest_rate']/100
    df3['days_to_maturity'] = df3['tau']
    df3['tau'] = df3['tau']/365
    df3['underlying_price'] = df3['BTC_price']
    df3['instrument_price'] = df3['option_price']
    df3['putcall'][df3['putcall']=='C']=1
    df3['putcall'][df3['putcall']=='P']=0
    
    df_1 = df3[['date','BTC_price', 'K', 'interest_rate', 'tau', 'putcall', 'IV',  'moneyness',
                'days_to_maturity',  'underlying_price', 'instrument_price']]
    df_1['mark_iv']=df_1['IV']/100
    
    df_1.to_csv('Results/option_' + datetime.datetime.strftime(curr_day_starttime, '%Y%m%d') + '.csv')
    
    conf_fname = prepare_confidence_band_data(df_1)
    
    rdate = base.as_Date(curr_day.strftime('%Y-%m-%d'))
    rdate_f = base.format(rdate, '%Y-%m-%d')
    
    tau = df['tau'].unique()[0]
    
    spd_btc = bootstrap(conf_fname, 'Data/BTC_USD_Quandl_2015_2022.csv', rdate_f, tau, simmethod, r, 
                                     'Results/Rookley_Q_CB_1722_1_2_5_ttm8', r_bandwidth, df_1['moneyness'].min(), df_1['moneyness'].max())
    
    try :
        
        spd_btc.to_csv('Results/Rookley_Q_CB_1722_1_2_5_ttm8/btc_pk_'  + str(curr_day_starttime.__format__('%Y-%m-%d')) +'_bw_' + str(r_bandwidth) +  '.csv')
        
        print(spd_btc)
        
    except Exception as e:
        error_i.append(i0)

Using Macos Locale


/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
R[write to console]: 
Attaching package: ‘matlab’


R[write to console]: The following object is masked from ‘package:stats’:

    reshape


R[write to console]: The following objects are masked from ‘package:utils’:

    find, fix


R[write to console]: The following object is masked from ‘package:base’:

    sum


R[write to console]: KernSmooth 2.23 loaded
Copyright M. P. Wand 1997-2009

2017-07-20 00:00:00


/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_51520/4253172800.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_51520/4253172800.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_51520/4253172800.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] "bandwidth: 0.3"
[1] "test no arbitrage condition"
[1] 44.694713  4.722982 66.258718
            m      spdy   spdy_up   spdy_dn           epk       ret   spd_ret  \
0    0.610137  0.836966  1.235413  0.438520  4.640089e+01 -0.494072  0.510664   
1    0.614010  0.838904  1.246118  0.431691  3.448678e+01 -0.487744  0.515096   
2    0.617883  0.840065  1.255983  0.424147  2.703575e+01 -0.481456  0.519062   
3    0.621756  0.840473  1.265035  0.415912  2.234995e+01 -0.475207  0.522570   
4    0.625629  0.840159  1.273305  0.407013  1.950870e+01 -0.468997  0.525628   
..        ...       ...       ...       ...           ...       ...       ...   
195  1.365395  0.200697  0.394107  0.007286  3.505508e+06  0.311443  0.274030   
196  1.369268  0.199426  0.387884  0.010968  1.265869e+07  0.314276  0.273067   
197  1.373141  0.198095  0.381655  0.014535  4.621919e+07  0.317101  0.272013   
198  1.377014  0.196700  0.375420  0.017980  1.776910e+08  0.319917  0.270858   
199  1.380887  0.195

...

[1] 44.694713  4.722982 66.258718
            m      spdy   spdy_up   spdy_dn          epk       ret   spd_ret  \
0    0.543466  0.018416  0.019242  0.017590     1.168344 -0.609788  0.010008   
1    0.547720  0.018725  0.019605  0.017844     0.924617 -0.601991  0.010256   
2    0.551974  0.019066  0.020002  0.018130     0.752768 -0.594254  0.010524   
3    0.556228  0.019442  0.020435  0.018448     0.629428 -0.586576  0.010814   
4    0.560483  0.019856  0.020908  0.018803     0.539502 -0.578957  0.011129   
..        ...       ...       ...       ...          ...       ...       ...   
195  1.373027  0.041823  0.055972  0.027674   124.649877  0.317018  0.057424   
196  1.377281  0.041486  0.054959  0.028014   229.744842  0.320112  0.057138   
197  1.381536  0.041175  0.053987  0.028363   435.179693  0.323196  0.056885   
198  1.385790  0.040884  0.053052  0.028717   851.848402  0.326270  0.056657   
199  1.390044  0.040609  0.052146  0.029072  1725.414761  0.329335  0.056449   

     

/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_51520/4253172800.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_51520/4253172800.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_51520/4253172800.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] 44.694713  4.722982 66.258718
            m      spdy   spdy_up   spdy_dn       epk       ret   spd_ret  \
0    0.586935  0.006653  0.008200  0.005106  0.091306 -0.532841  0.003905   
1    0.590742  0.006830  0.008429  0.005232  0.088945 -0.526376  0.004035   
2    0.594548  0.007037  0.008684  0.005390  0.087878 -0.519953  0.004184   
3    0.598355  0.007274  0.008967  0.005580  0.088078 -0.513571  0.004352   
4    0.602161  0.007543  0.009280  0.005806  0.089564 -0.507230  0.004542   
..        ...       ...       ...       ...       ...       ...       ...   
195  1.329206  0.017237  0.036372 -0.001898  0.614421  0.284582  0.022911   
196  1.333013  0.015728  0.035513 -0.004057  0.719235  0.287442  0.020966   
197  1.336819  0.014283  0.034699 -0.006132  0.858995  0.290293  0.019094   
198  1.340626  0.012899  0.033925 -0.008126  1.046386  0.293137  0.017293   
199  1.344433  0.011574  0.033190 -0.010042  1.299370  0.295972  0.015560   

       cb_ret_up    cb_ret_dn      epk_re

/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_51520/4253172800.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_51520/4253172800.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_51520/4253172800.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] 44.694713  4.722982 66.258718
            m      spdy   spdy_up   spdy_dn        epk       ret   spd_ret  \
0    0.715536  0.058426  0.067121  0.049731   0.355395 -0.334723  0.041806   
1    0.718814  0.060958  0.069649  0.052268   0.353384 -0.330153  0.043818   
2    0.722092  0.063673  0.072341  0.055005   0.350677 -0.325603  0.045978   
3    0.725369  0.066582  0.075209  0.057955   0.348018 -0.321075  0.048297   
4    0.728647  0.069697  0.078264  0.061131   0.346059 -0.316566  0.050785   
..        ...       ...       ...       ...        ...       ...       ...   
195  1.354669  0.020409  0.023855  0.016962   6.384961  0.303557  0.027647   
196  1.357946  0.019650  0.023364  0.015935   8.844811  0.305974  0.026683   
197  1.361224  0.018890  0.022881  0.014900  12.477730  0.308384  0.025714   
198  1.364502  0.018129  0.022401  0.013857  17.924324  0.310789  0.024737   
199  1.367779  0.017365  0.021921  0.012809  26.214410  0.313188  0.023751   

     cb_ret_up  cb_ret_dn   e

In [2]:
print(error_i)

[1, 2, 3, 6, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 31, 32, 36, 37, 38, 43, 55, 107, 224]


In [10]:
import platform
plat = platform.system()
import locale
if plat == 'Darwin':
    print('Using Macos Locale')
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
else:
    locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import datetime
import pandas as pd
import sys
import matplotlib.pyplot as plt
import csv
import gc
import math
import numpy as np
import os
import pdb
import pickle
import statsmodels.api as sm
import time


import rpy2.robjects.packages as rpackages
from rpy2 import robjects

from scipy import stats
from scipy.integrate import simps
from scipy import interpolate

from src.helpers import save_dict, load_dict
from src.brc import BRC
from src.strategies import IronCondor
from src.quandl_data import get_btc_prices_2015_2022

from numpy.lib.function_base import insert

from statsmodels.nonparametric.kernel_regression import KernelReg as nadarajawatson

from sklearn.neighbors import KernelDensity

from pathlib import Path

def filter_sub(_df,  mindate, maxdate, tau):
    # Subset

    # Only calls, tau in [0, 0.25] and fix one day (bc looking at intra day here)
    # (_df['is_call'] == 1) & --> Dont need to filter for calls because that happens in the R script
    # Also need to consider put-call parity there
    sub = _df[(_df['date'] >= mindate) & (_df['date'] <= maxdate) & (_df['moneyness'] >= 0.5) & (_df['moneyness'] < 2) &(_df['mark_iv'] > 0)]# &

    if tau > 0:
        sub = sub[sub['tau'] == tau]

    nrows = sub.shape[0]
    if nrows == 0:
        raise(ValueError('Sub is empty'))
   
    sub['moneyness'] = round(sub['moneyness'], 3)
    sub['underlying_price'] = round(sub['underlying_price'], 2)

    sub = sub.drop_duplicates()
    
    print(sub.describe())

    if nrows > 50000:
        print('large df, subsetting')
        sub = sub.sample(10000)
        print(sub.describe())

    return sub

def prepare_confidence_band_data(df):

            # Old
                # Save for Confidence Band 
                    # Column 1 - Day
                    # Column 2 - Month
                    # Column 3 - Year
                    # Column 4 - Call Indicator
                    # Column 5 - Time Maturity in Days
                    # Column 6 - Strike
                    # Column 7 - IV
                    # Column 8 - Spot
                    # Column 9 - Risk Free Rate
                    # Column 10 - 
                    # Column 11 - Forward
                # 3	1	1997	0	49	3025	165.8	2861.706	0.031465	0.150026951	2863.26
                # Day Month Year Put Days Strike IV     Spot         Risk Free    Tau in Days (255)       Forward

    """
    # Mapping Table for Input

    # Column ... X
    # 1 - date
    # 2 - IV
    # 3 - 1 for Call, 0 for Put
    # 4 - Tau 
    # 5 - Strike
    # 6 - assigned meanwhile as put2call[,6]+ put2call[,7] -  put2call[,5]*exp(- mean(put2call[,8])* tau);
    # 7 - Spot
    # 8 - Probably Interest Rate
    # 9 - assigned meanwhile Moneyness: Spot/Strike
    """
    b = df[['date', 'mark_iv', 'putcall', 'tau', 'K', 'underlying_price', 'interest_rate', 'moneyness', 'instrument_price']]
    b['placeholder'] = 0
    b = b.reset_index(drop=True)
    # b['interest_rate'] = 0.00
    b['forward'] = 1

    # Reorder
    df_c = b[['date', 'mark_iv', 'putcall', 'tau', 'K', 'instrument_price', 'underlying_price', 'interest_rate', 'moneyness']]

    unique_maturities = df.days_to_maturity.unique()[0]
    fname = str('confidence_band_input_') + str(unique_maturities) + str('.csv')
    df_c.to_csv(fname, index = False)
    return fname

def bootstrap(conf_fname, phys_fname, rdate, tau, simmethod, r, out_dir, bw, lower, upper):
                            
    try:
        # MAKE SURE TO USE DIFFERENT 2ND INPUT FOR SP500!!
        moneyness, spd, epk, epk_lo, epk_up, logret, spd_logret, epk_logret, epk_logret_lo, epk_logret_up, epk_BS, tau, volas, spd_lo, spd_up, cdf_m, cdf_ret= r.bootstrap_epk(robjects.StrVector([conf_fname]), 
                                      robjects.StrVector([phys_fname]),
                                      robjects.StrVector(rdate),
                                      robjects.StrVector([out_dir]),
                                      robjects.FloatVector([bw]),
                                      robjects.FloatVector([lower]),
                                      robjects.FloatVector([upper])
                                     )
        # Can recover PD here as SPD/EPK
        spd_df = pd.DataFrame({'m': moneyness,
                               'spdy': spd,
                               'spdy_up': spd_up,
                               'spdy_dn': spd_lo,
                               'epk': epk,
                               'ret': logret,
                               'spd_ret': spd_logret,
                               'cb_ret_up': epk_logret_up,
                               'cb_ret_dn': epk_logret_lo,
                               'epk_ret': epk_logret,
                               'volatility': volas,
                               'epk_up': epk_up,
                               'epk_dn': epk_lo,
                               'cdf_m': cdf_m,
                               'cdf_ret': cdf_ret})

        return spd_df
    
    except Exception as e:
        print('Sim failed, proceeding with the next one... - ', simmethod)
        print('exception: ', e)
        
# Initiate R Objects
base = rpackages.importr("base") 
r = robjects.r
r.source('src/confidence_bands.R')
r.source('src/Q_Rookley_main_3.R')

# BTC prices
#tdat = get_btc_prices_2015_2022()
#tdat[::-1].to_csv('Data/BTC_USD_Quandl_2015_2022.csv', index = False)

bw_range = np.array([0.3])
r_bandwidth = bw_range[0]

simmethod = ['oldschool']

df = pd.read_csv('Data/20172022_processed_1_3_4.csv')
interest_rate_data = pd.read_csv('Data/IR_daily.csv')
interest_rate_data = interest_rate_data.rename(columns={'index': 'date','DTB3': 'interest_rate'})

df1=df[df['tau']==8]
dates=df1['date'].unique()
#df1=df[df['date']>=dates[9]]
#dates = df1['date'].unique()

dates1 = dates

error_i = []

Using Macos Locale


In [12]:
i0=62
df2 = df1[df1['date']==dates1[i0]]
    
curr_day = datetime.datetime.strptime(dates1[i0],'%Y-%m-%d')
curr_day_starttime = curr_day.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
curr_day_endtime = curr_day.replace(hour = 23, minute = 59, second = 59, microsecond = 0)
print(curr_day)

df3 = pd.merge(df2, interest_rate_data, on='date', how='left')
    

df3['interest_rate'] = df3['interest_rate']/100
df3['days_to_maturity'] = df3['tau']
df3['tau'] = df3['tau']/365
df3['underlying_price'] = df3['BTC_price']
df3['instrument_price'] = df3['option_price']
df3['putcall'][df3['putcall']=='C']=1
df3['putcall'][df3['putcall']=='P']=0
    
df_1 = df3[['date','BTC_price', 'K', 'interest_rate', 'tau', 'putcall', 'IV',  'moneyness',
                'days_to_maturity',  'underlying_price', 'instrument_price']]
df_1['mark_iv']=df_1['IV']/100
    
df_1.to_csv('Results/option_' + datetime.datetime.strftime(curr_day_starttime, '%Y%m%d') + '.csv')
    
conf_fname = prepare_confidence_band_data(df_1)
    
rdate = base.as_Date(curr_day.strftime('%Y-%m-%d'))
rdate_f = base.format(rdate, '%Y-%m-%d')
    
tau = df['tau'].unique()[0]
    
spd_btc = bootstrap(conf_fname, 'Data/BTC_USD_Quandl_2015_2022.csv', rdate_f, tau, simmethod, r, 
                                     'Results/Rookley_Q_CB_1722_1_2_5_ttm8', r_bandwidth, df_1['moneyness'].min(), df_1['moneyness'].max())
    

try :
    spd_btc.to_csv('Results/Rookley_Q_CB_1722_1_2_5_ttm8/btc_pk_'  + str(curr_day_starttime.__format__('%Y-%m-%d')) +'_bw_' + str(r_bandwidth) +  '.csv')
    print(spd_btc)
        
except Exception as e:

    
    
    error_i.append(i0)

2019-05-09 00:00:00
[1] "bandwidth: 0.3"
[1] "test no arbitrage condition"


/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_12734/3306296290.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_12734/3306296290.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_12734/3306296290.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

[1] 44.694713  4.722982 66.258718
                [,1]      [,2]        [,3]        [,4]
sigmas -0.1985137970 0.8452294 -2.74175688 17.83268884
       -0.1963584632 0.8403849 -2.70879943 17.74785510
       -0.1942077648 0.8356006 -2.67602992 17.66367937
       -0.1920616820 0.8308763 -2.64344823 17.58019259
       -0.1899201950 0.8262116 -2.61105408 17.49742427
       -0.1877832842 0.8216060 -2.57884703 17.41540243
       -0.1856509301 0.8170592 -2.54682643 17.33415364
       -0.1835231132 0.8125708 -2.51499150 17.25370301
       -0.1813998143 0.8081405 -2.48334127 17.17407427
       -0.1792810142 0.8037678 -2.45187466 17.09528966
       -0.1771666940 0.7994523 -2.42059041 17.01737007
       -0.1750568347 0.7951937 -2.38948713 16.94033495
       -0.1729514175 0.7909916 -2.35856329 16.86420237
       -0.1708504238 0.7868456 -2.32781724 16.78898903
       -0.1687538350 0.7827553 -2.29724721 16.71471023
       -0.1666616327 0.7787202 -2.26685131 16.64137992
       -0.1645737986 0.7747401 

        0.0769760731 0.6481907  0.82788062  6.82875644
        0.0786128273 0.6497399  0.83932447  6.62893724
        0.0802469069 0.6513088  0.85031117  6.42515308
        0.0818783207 0.6528965  0.86083020  6.21736287
        0.0835070772 0.6545018  0.87087090  6.00552576
        0.0851331853 0.6561239  0.88042250  5.78960110
        0.0867566534 0.6577616  0.88947415  5.56954849
        0.0883774901 0.6594139  0.89801486  5.34532773
        0.0899957040 0.6610797  0.90603356  5.11689888
        0.0916113034 0.6627579  0.91351904  4.88422222
        0.0932242969 0.6644473  0.92046002  4.64725825
        0.0948346929 0.6661468  0.92684509  4.40596773
        0.0964424997 0.6678551  0.93266274  4.16031167
        0.0980477255 0.6695711  0.93790135  3.91025129
        0.0996503788 0.6712934  0.94254921  3.65574811
        0.1012504676 0.6730208  0.94659450  3.39676390
        0.1028480003 0.6747521  0.95002529  3.13326068
        0.1044429849 0.6764857  0.95282955  2.86520078
        0.